# Check Guild

In [1]:
import os, sys, random
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.contrib.auth import get_user_model

In [6]:
import emporium.models as EM
import engine.models as GM

In [7]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Guild Properties

In [8]:
userMd = get_user_model().objects.filter(user_name='Admin')[0]
userMd

<User: admin@thieves-guild.com>

In [9]:
guildMd = GM.Guild.objects.GetOrNone(UserFK=userMd, Selected=True)
PD.DataFrame([guildMd.__dict__]).drop(['id', '_state'], axis=1, errors='ignore')

,UserFK_id,Name,LastPlayed,CreateDate,Selected,VaultGold,VaultStone,VaultGems,LastHeist,CampaignWorld,DungeonCheck
0,1,411,2025-01-26,2025-01-18,True,1093,1400,47,campaign,2,2025-01-26


In [10]:
thieves = GM.ThiefInGuild.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(thieves).drop(['GuildFK_id',
        'Name', 'BasePower', 'Experience', 
        #'Position', 'Wounds', 'Cooldown',
        'BaseAgi', 'BaseCun', 'BaseMig', 'BaseEnd', 
#         'TrainedAgi', 'TrainedCun', 'TrainedMig', 'TrainedEnd', 
        'Attack', 'Damage', 'Defense',
        'Traverse', 'Sabotage', 'Perceive', 
        ], axis=1, errors='ignore')

,id,Class,Stars,Level,PowerBase,TrainedAgi,TrainedCun,TrainedMig,TrainedEnd,TrainedSkills,Power,Agility,Cunning,Might,Endurance,Health,Status,CooldownExpire
0,116,Burglar,1,1,62,0,0,0,0,[],150,5,0,0,0,75,Ready,NaT
1,117,Scoundrel,1,1,62,0,0,0,0,[],114,0,5,0,0,75,Ready,NaT
2,118,Ruffian,1,2,62,0,0,0,1,[],128,0,0,5,1,80,Ready,NaT
3,119,Burglar,1,2,62,0,0,0,0,[dmg 2],128,5,0,0,0,75,Ready,NaT
4,120,Scoundrel,1,3,62,0,0,1,0,[att 2],118,0,4,1,0,75,Beaten,2025-01-26 16:48:09+00:00
5,121,Ruffian,1,2,62,0,0,0,0,[att 2],128,0,0,5,0,75,Ready,NaT


In [11]:
thiefMds = GM.ThiefInGuild.objects.filter(GuildFK=guildMd)
thiefLs = []

for md in thiefMds:

    thiefDx = md.__dict__
    thiefLs.append(thiefDx)
    
    thiefItems = GM.ItemInGuild.objects.filter(ThiefFK=md).values()
    thiefDx['ItemCount'] = len(thiefItems)
    
    slots = ['weapon', 'armor', 'head', 'hands', 'feet', ]
    
    for sl in slots:
        itemCheck = [x for x in thiefItems if x['Slot']==sl]
        
        if itemCheck:
            item = itemCheck[0]
            res = item.pop('GuildFK_id')
            res = item.pop('ThiefFK_id')
            if sl in ['weapon', 'armor']: stat = item['Trait'][:3]
            else:     stat = 'skl' if item['Skill'] else 'cmb'
            item['icon'] = f"{sl}-{stat}.png"
        else:
            item = {}
            item['id'] = -1
            item['icon'] = f"{sl}-empty.png"

        thiefDx[sl] = item


PD.DataFrame(thiefItems)

,id,Throne,Name,Slot,MagicLv,TotalLv,Power,Requirement,Trait,Skill,Combat,Magic,icon
0,717,1,Sword & Shield,weapon,0,2,26,Ruffian,mig 1,None,att 3,None,weapon-mig.png
1,718,1,Chain Maille,armor,0,2,26,Ruffian,mig 1,None,def 1,None,armor-mig.png


In [12]:
thiefDf = PD.DataFrame(thiefLs)
thiefDf = thiefDf.drop(['_state', 'GuildFK_id', 'BasePower', 'BaseAgi', 'BaseCun', 'BaseMig', 'BaseEnd', 
                        'TrainedAgi', 'TrainedCun', 'TrainedMig', 'TrainedEnd',
                        'Experience', 'Position', 'Wounds', 'Cooldown'], 
                    axis=1, errors='ignore')
thiefDf = thiefDf.sort_values('Power', ascending=False)
thiefDf

,id,Name,Class,Stars,Level,PowerBase,TrainedSkills,Power,Agility,Cunning,...,Perceive,Traverse,Status,CooldownExpire,ItemCount,weapon,armor,head,hands,feet
0,116,Furthur,Burglar,1,1,62,[],150,5,0,...,3,3,Ready,NaT,5,"{'id': 707, 'Throne': 1, 'Name': 'Twin Daggers...","{'id': 708, 'Throne': 1, 'Name': 'Brigandine',...","{'id': 719, 'Throne': 1, 'Name': 'Cowl', 'Slot...","{'id': 721, 'Throne': 1, 'Name': 'Gloves', 'Sl...","{'id': 723, 'Throne': 1, 'Name': 'Boots', 'Slo..."
2,118,Lancet,Ruffian,1,2,62,[],128,0,0,...,0,0,Ready,NaT,2,"{'id': 711, 'Throne': 1, 'Name': 'Sword & Shie...","{'id': 712, 'Throne': 1, 'Name': 'Chain Maille...","{'id': -1, 'icon': 'head-empty.png'}","{'id': -1, 'icon': 'hands-empty.png'}","{'id': -1, 'icon': 'feet-empty.png'}"
3,119,Belgrath,Burglar,1,2,62,[dmg 2],128,5,0,...,0,0,Ready,NaT,2,"{'id': 713, 'Throne': 1, 'Name': 'Twin Daggers...","{'id': 714, 'Throne': 1, 'Name': 'Brigandine',...","{'id': -1, 'icon': 'head-empty.png'}","{'id': -1, 'icon': 'hands-empty.png'}","{'id': -1, 'icon': 'feet-empty.png'}"
5,121,Vaison,Ruffian,1,2,62,[att 2],128,0,0,...,0,0,Ready,NaT,2,"{'id': 717, 'Throne': 1, 'Name': 'Sword & Shie...","{'id': 718, 'Throne': 1, 'Name': 'Chain Maille...","{'id': -1, 'icon': 'head-empty.png'}","{'id': -1, 'icon': 'hands-empty.png'}","{'id': -1, 'icon': 'feet-empty.png'}"
4,120,Gertelheim,Scoundrel,1,3,62,[att 2],118,0,4,...,0,0,Beaten,2025-01-26 16:48:09+00:00,1,"{'id': 715, 'Throne': 1, 'Name': 'Blackthorn',...","{'id': -1, 'icon': 'armor-empty.png'}","{'id': -1, 'icon': 'head-empty.png'}","{'id': -1, 'icon': 'hands-empty.png'}","{'id': -1, 'icon': 'feet-empty.png'}"
1,117,Wulfgard,Scoundrel,1,1,62,[],114,0,5,...,0,0,Ready,NaT,2,"{'id': 709, 'Throne': 1, 'Name': 'Blackthorn',...","{'id': 710, 'Throne': 1, 'Name': 'Vestment', '...","{'id': -1, 'icon': 'head-empty.png'}","{'id': -1, 'icon': 'hands-empty.png'}","{'id': -1, 'icon': 'feet-empty.png'}"


In [13]:
guildItems = GM.ItemInGuild.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(guildItems).drop(['id'], axis=1, errors='ignore').sort_values('Slot')

,GuildFK_id,ThiefFK_id,Throne,Name,Slot,MagicLv,TotalLv,Power,Requirement,Trait,Skill,Combat,Magic
1,41,116.0,1,Brigandine,armor,0,2,26,Burglar,agi 1,None,def 3,None
3,41,117.0,1,Vestment,armor,0,2,26,Scoundrel,cun 1,None,def 2,None
5,41,118.0,1,Chain Maille,armor,0,2,26,Ruffian,mig 1,None,def 1,None
7,41,119.0,1,Brigandine,armor,0,2,26,Burglar,agi 1,None,def 3,None
9,41,NaN,1,Vestment,armor,0,2,26,Scoundrel,cun 1,None,def 2,None
11,41,121.0,1,Chain Maille,armor,0,2,26,Ruffian,mig 1,None,def 1,None
17,41,NaN,1,Greaves,feet,0,1,12,None,None,None,att 1,None
16,41,116.0,1,Boots,feet,0,1,12,None,None,tra 3,None,None
15,41,NaN,1,Gauntlet,hands,0,1,12,None,None,None,dmg 1,None
14,41,116.0,1,Gloves,hands,0,1,12,None,None,sab 3,None,None


### Stages

In [14]:
towers = GM.GuildTower.objects.all().values()
PD.DataFrame(towers).drop(['id'], axis=1, errors='ignore')

AttributeError: module 'engine.models' has no attribute 'GuildTower'

### Stores

In [ ]:
unlocks = GM.ThiefUnlocked.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(unlocks).drop(['id'], axis=1, errors='ignore')

In [ ]:
unlocks = GM.ItemUnlocked.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(unlocks).drop(['id'], axis=1, errors='ignore')